In [1]:
import numpy as np
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Conv1D, MaxPooling1D, Dense, Flatten, LSTM, Bidirectional
from keras.models import Sequential
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# 選擇數據處理模式：'N-R' 或 'R'
mode = 'N-R'  # 可以選擇 'N-R' 或 'R'

# 載入與預處理數據
def load_semeval_data(file_paths):
    data = []
    labels = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split('\t')
                if len(parts) == 3:
                    data.append(parts[2])
                    labels.append(parts[1])
    return pd.DataFrame({'tweet': data, 'label': labels})

file_paths = [
    './dataset/train/twitter-2013train-A.txt',
    './dataset/train/twitter-2015train-A.txt',
    './dataset/train/twitter-2016train-A.txt',
]

dataset = load_semeval_data(file_paths)

# 預處理推文文本
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"https?:\/\/\S+", "", tweet)
    tweet = re.sub(r"[^a-zA-Z0-9\s]", "", tweet)
    return tweet

dataset['tweet'] = dataset['tweet'].apply(preprocess_tweet)

# Tokenization 和 Padding 設置
MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 40  # 用於 N-R 模式
REGION_SIZE = 10  # 每個區域的長度，用於 R 模式
NUM_REGIONS = 4  # 區域數量，用於 R 模式
EMBEDDING_DIM = 25

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(dataset['tweet'].values)
word_index = tokenizer.word_index

# 根據模式選擇不同的數據處理方法
if mode == 'N-R':
    # Non-Regional 模式，直接將推文轉為固定長度序列
    X = tokenizer.texts_to_sequences(dataset['tweet'].values)
    X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
    input_length = MAX_SEQUENCE_LENGTH  # 設置模型的 input_length
elif mode == 'R':
    # Regional 模式，將推文分成多個區域並填充至相同長度
    X = tokenizer.texts_to_sequences(dataset['tweet'].values)
    X_padded = []
    for tweet in X:
        tweet_regions = []
        for i in range(NUM_REGIONS):
            start = i * REGION_SIZE
            end = start + REGION_SIZE
            region = tweet[start:end]
            if len(region) < REGION_SIZE:
                region = region + [0] * (REGION_SIZE - len(region))
            tweet_regions.extend(region)
        X_padded.append(tweet_regions)
    X = np.array(X_padded)
    input_length = NUM_REGIONS * REGION_SIZE  # 設置模型的 input_length
else:
    raise ValueError("Invalid mode. Choose 'N-R' or 'R'.")

# 標籤處理
label_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}
y = dataset['label'].map(label_mapping).values
y = pd.get_dummies(y).values

# 數據分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 載入 GloVe 詞嵌入
embeddings_index = {}
with open('./dataset/glove.twitter.27B.25d.txt', encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# 定義嵌入層，用於後續模型
embedding_layer = Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                            input_length=input_length, trainable=False)

In [2]:
from keras.layers import Flatten

# 建立單一 CNN 模型並加入 Flatten 層
cnn_model = Sequential()
cnn_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH, trainable=False))
cnn_model.add(Conv1D(filters=12, kernel_size=3, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=3))
cnn_model.add(Flatten())  # 加入 Flatten 層
cnn_model.add(Dense(3, activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 2s 3ms/step - loss: 1.0066 - accuracy: 0.4761 - val_loss: 0.9570 - val_accuracy: 0.5082
Epoch 2/10
405/405 [==============================] - 1s 3ms/step - loss: 0.9321 - accuracy: 0.5369 - val_loss: 0.9162 - val_accuracy: 0.5338
Epoch 3/10
405/405 [==============================] - 1s 2ms/step - loss: 0.9009 - accuracy: 0.5555 - val_loss: 0.8992 - val_accuracy: 0.5391
Epoch 4/10
405/405 [==============================] - 1s 2ms/step - loss: 0.8847 - accuracy: 0.5651 - val_loss: 0.9071 - val_accuracy: 0.5478
Epoch 5/10
405/405 [==============================] - 1s 2ms/step - loss: 0.8729 - accuracy: 0.5729 - val_loss: 0.8992 - val_accuracy: 0.5564
Epoch 6/10
405/405 [==============================] - 1s 2ms/step - loss: 0.8643 - accuracy: 0.5805 - val_loss: 0.8911 - val_accuracy: 0.5617
Epoch 7/10
405/405 [==============================] - 1s 2ms/step - loss: 0.8560 - accuracy: 0.5829 - val_loss: 0.8884 - val_accuracy: 0.5604
Epoch 

In [3]:
# 單一 LSTM 模型 (Single LSTM)

from keras.layers import LSTM

# 建立單一 LSTM 模型
lstm_model = Sequential()
lstm_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                         input_length=MAX_SEQUENCE_LENGTH, trainable=False))
lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(3, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 25s 56ms/step - loss: 0.9247 - accuracy: 0.5328 - val_loss: 0.8803 - val_accuracy: 0.5654
Epoch 2/10
405/405 [==============================] - 22s 55ms/step - loss: 0.8870 - accuracy: 0.5627 - val_loss: 0.8698 - val_accuracy: 0.5731
Epoch 3/10
405/405 [==============================] - 22s 53ms/step - loss: 0.8755 - accuracy: 0.5666 - val_loss: 0.8447 - val_accuracy: 0.5904
Epoch 4/10
405/405 [==============================] - 21s 53ms/step - loss: 0.8601 - accuracy: 0.5788 - val_loss: 0.8433 - val_accuracy: 0.5904
Epoch 5/10
405/405 [==============================] - 21s 53ms/step - loss: 0.8495 - accuracy: 0.5871 - val_loss: 0.8389 - val_accuracy: 0.5932
Epoch 6/10
405/405 [==============================] - 22s 55ms/step - loss: 0.8388 - accuracy: 0.5973 - val_loss: 0.8314 - val_accuracy: 0.6080
Epoch 7/10
405/405 [==============================] - 23s 56ms/step - loss: 0.8291 - accuracy: 0.5960 - val_loss: 0.8264 - val_accuracy:

In [4]:
# CNN + LSTM 組合模型 (CNN + LSTM Combined Network)

# 建立 CNN + LSTM 模型
cnn_lstm_model = Sequential()
cnn_lstm_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                             input_length=MAX_SEQUENCE_LENGTH, trainable=False))
cnn_lstm_model.add(Conv1D(filters=12, kernel_size=3, activation='relu'))
cnn_lstm_model.add(MaxPooling1D(pool_size=3))
cnn_lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
cnn_lstm_model.add(Dense(3, activation='softmax'))
cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 12s 23ms/step - loss: 0.9646 - accuracy: 0.5009 - val_loss: 0.9118 - val_accuracy: 0.5369
Epoch 2/10
405/405 [==============================] - 8s 21ms/step - loss: 0.9107 - accuracy: 0.5500 - val_loss: 0.8943 - val_accuracy: 0.5533
Epoch 3/10
405/405 [==============================] - 8s 21ms/step - loss: 0.8944 - accuracy: 0.5584 - val_loss: 0.8871 - val_accuracy: 0.5583
Epoch 4/10
405/405 [==============================] - 8s 21ms/step - loss: 0.8803 - accuracy: 0.5635 - val_loss: 0.8741 - val_accuracy: 0.5713
Epoch 5/10
405/405 [==============================] - 9s 21ms/step - loss: 0.8735 - accuracy: 0.5733 - val_loss: 0.8692 - val_accuracy: 0.5691
Epoch 6/10
405/405 [==============================] - 8s 21ms/step - loss: 0.8643 - accuracy: 0.5768 - val_loss: 0.8669 - val_accuracy: 0.5719
Epoch 7/10
405/405 [==============================] - 8s 21ms/step - loss: 0.8561 - accuracy: 0.5822 - val_loss: 0.8615 - val_accuracy: 0.576

In [5]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# CNN 特徵提取模型
cnn_feature_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=12, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=3),
    Flatten()
])

# LSTM 特徵提取模型
lstm_feature_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Flatten()
])

# 提取 CNN 和 LSTM 特徵
cnn_features = cnn_feature_model.predict(X_train)
lstm_features = lstm_feature_model.predict(X_train)
combined_features = np.hstack((cnn_features, lstm_features))

# 使用 SVM 進行分類
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(combined_features, np.argmax(y_train, axis=1))

# 評估
cnn_test_features = cnn_feature_model.predict(X_test)
lstm_test_features = lstm_feature_model.predict(X_test)
combined_test_features = np.hstack((cnn_test_features, lstm_test_features))
y_pred = svm_classifier.predict(combined_test_features)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred))

102/102 [==============================] - 1s 12ms/step
              precision    recall  f1-score   support

           0       0.55      0.65      0.60      1372
           1       1.00      0.00      0.00       466
           2       0.54      0.62      0.58      1397

    accuracy                           0.55      3235
   macro avg       0.70      0.43      0.39      3235
weighted avg       0.61      0.55      0.50      3235



In [6]:
# 單一 3 層 CNN 和 LSTM 網絡

three_layer_cnn_lstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

three_layer_cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
three_layer_cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = three_layer_cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 6s 10ms/step - loss: 0.9541 - accuracy: 0.5107 - val_loss: 0.9096 - val_accuracy: 0.5363
Epoch 2/10
405/405 [==============================] - 4s 10ms/step - loss: 0.8961 - accuracy: 0.5531 - val_loss: 0.8834 - val_accuracy: 0.5586
Epoch 3/10
405/405 [==============================] - 4s 10ms/step - loss: 0.8630 - accuracy: 0.5836 - val_loss: 0.8754 - val_accuracy: 0.5604
Epoch 4/10
405/405 [==============================] - 4s 10ms/step - loss: 0.8378 - accuracy: 0.6022 - val_loss: 0.8803 - val_accuracy: 0.5638
Epoch 5/10
405/405 [==============================] - 4s 10ms/step - loss: 0.8061 - accuracy: 0.6177 - val_loss: 0.8645 - val_accuracy: 0.5651
Epoch 6/10
405/405 [==============================] - 4s 10ms/step - loss: 0.7691 - accuracy: 0.6398 - val_loss: 0.8768 - val_accuracy: 0.5728
Epoch 7/10
405/405 [==============================] - 4s 10ms/step - loss: 0.7318 - accuracy: 0.6623 - val_loss: 0.9045 - val_accuracy: 0.5731

In [7]:
# 多重 CNN 和 LSTM 網絡

multi_cnn_lstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

multi_cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
multi_cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = multi_cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 9s 18ms/step - loss: 0.9534 - accuracy: 0.5128 - val_loss: 0.8931 - val_accuracy: 0.5558
Epoch 2/10
405/405 [==============================] - 7s 17ms/step - loss: 0.8892 - accuracy: 0.5556 - val_loss: 0.8737 - val_accuracy: 0.5641
Epoch 3/10
405/405 [==============================] - 7s 17ms/step - loss: 0.8608 - accuracy: 0.5815 - val_loss: 0.8614 - val_accuracy: 0.5793
Epoch 4/10
405/405 [==============================] - 7s 17ms/step - loss: 0.8356 - accuracy: 0.5985 - val_loss: 0.9396 - val_accuracy: 0.5249
Epoch 5/10
405/405 [==============================] - 8s 19ms/step - loss: 0.8109 - accuracy: 0.6116 - val_loss: 0.8546 - val_accuracy: 0.5901
Epoch 6/10
405/405 [==============================] - 7s 17ms/step - loss: 0.7742 - accuracy: 0.6354 - val_loss: 0.8678 - val_accuracy: 0.5886
Epoch 7/10
405/405 [==============================] - 7s 17ms/step - loss: 0.7434 - accuracy: 0.6561 - val_loss: 0.8686 - val_accuracy: 0.5867

In [8]:
from keras.layers import Bidirectional

# 單一 3 層 CNN 和雙向 LSTM 網絡

three_layer_cnn_bilstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(3, activation='softmax')
])

three_layer_cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
three_layer_cnn_bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = three_layer_cnn_bilstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 11s 17ms/step - loss: 0.9557 - accuracy: 0.5097 - val_loss: 0.9154 - val_accuracy: 0.5376
Epoch 2/10
405/405 [==============================] - 6s 15ms/step - loss: 0.8946 - accuracy: 0.5582 - val_loss: 0.8858 - val_accuracy: 0.5660
Epoch 3/10
405/405 [==============================] - 6s 15ms/step - loss: 0.8637 - accuracy: 0.5815 - val_loss: 0.8708 - val_accuracy: 0.5759
Epoch 4/10
405/405 [==============================] - 6s 15ms/step - loss: 0.8342 - accuracy: 0.5962 - val_loss: 0.8624 - val_accuracy: 0.5849
Epoch 5/10
405/405 [==============================] - 6s 16ms/step - loss: 0.8057 - accuracy: 0.6151 - val_loss: 0.8729 - val_accuracy: 0.5743
Epoch 6/10
405/405 [==============================] - 7s 16ms/step - loss: 0.7657 - accuracy: 0.6390 - val_loss: 0.8727 - val_accuracy: 0.5734
Epoch 7/10
405/405 [==============================] - 7s 17ms/step - loss: 0.7217 - accuracy: 0.6654 - val_loss: 0.9257 - val_accuracy: 0.566

In [9]:
# 多重 CNN 和雙向 LSTM 網絡

multi_cnn_bilstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(3, activation='softmax')
])

multi_cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
multi_cnn_bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = multi_cnn_bilstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 18s 33ms/step - loss: 0.9489 - accuracy: 0.5090 - val_loss: 0.8947 - val_accuracy: 0.5474
Epoch 2/10
405/405 [==============================] - 13s 32ms/step - loss: 0.8897 - accuracy: 0.5668 - val_loss: 0.8779 - val_accuracy: 0.5663
Epoch 3/10
405/405 [==============================] - 14s 34ms/step - loss: 0.8603 - accuracy: 0.5818 - val_loss: 0.8618 - val_accuracy: 0.5731
Epoch 4/10
405/405 [==============================] - 12s 31ms/step - loss: 0.8278 - accuracy: 0.6060 - val_loss: 0.8543 - val_accuracy: 0.5740
Epoch 5/10
405/405 [==============================] - 13s 31ms/step - loss: 0.7993 - accuracy: 0.6199 - val_loss: 0.8471 - val_accuracy: 0.5889
Epoch 6/10
405/405 [==============================] - 13s 32ms/step - loss: 0.7652 - accuracy: 0.6397 - val_loss: 0.8504 - val_accuracy: 0.5910
Epoch 7/10
405/405 [==============================] - 13s 32ms/step - loss: 0.7269 - accuracy: 0.6664 - val_loss: 0.8666 - val_accuracy: